<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment3/Assignment3_E2_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import keras
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import json

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

from __future__ import absolute_import, division, print_function
keras.__version__

'2.2.4'

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 3s 0us/step


In [3]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [4]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

#define decoding function
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

#display first review in words
decode_review(train_data[0])

1646592/1641221 [==============================] - 1s 1us/step


"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [0]:
#make the length of each review array as same
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
!ls '/content/drive/My Drive'

'6210 Database Design.gdoc'	   out_data_train_1.zip
 7374Assignment3		   out_data_train.zip
 aclImdb			   out_data_val
 AED_Lab1_Part1.mp4		   out_data_val_1.zip
 AED_Lab1_Part2Final.mp4	   out_data_val.zip
 Assignment2-E1.ipynb		   resized-tiny-imagenet-200
 Assignment3.json		   resized-tiny-imagenet-200.zip
'Colab Notebooks'		   Resume0306.gdoc
 data				   RL-NikBrown.gdoc
 drive				   Track-guidelines.pdf
 Final_test.json		   val_data
 Final_train.json		   Video_Games_Sales.csv
 games.zip			  'VideoGames-Version2 2.zip'
 Imagenet32_train		   VideoGames-Version4-2.zip
'NBA_stats_f (Recovered).gsheet'   VideoGames-Version4.zip
 out_data_train			   y_val_1.npy
 out_data_train_1


In [15]:
os.chdir(os.path.join(os.getcwd(),'drive/My Drive'))
print(os.getcwd())

/content/drive/My Drive


In [0]:
import urllib.request
import tarfile

In [19]:
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz" #download url
filepath="7374Assignment3/aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
  result=urllib.request.urlretrieve(url,filepath)
  print('downloaded:',result)

downloaded: ('7374Assignment3/aclImdb_v1.tar.gz', <http.client.HTTPMessage object at 0x7f921b4aac50>)


In [0]:
if not os.path.exists("7374Assignment3/aclImdb"):
  tfile = tarfile.open("7374Assignment3/aclImdb_v1.tar.gz", 'r:gz')
  result=tfile.extractall('7374Assignment3/')

In [22]:
from sklearn.datasets import load_files
reviews_train = load_files("7374Assignment3/aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target

print("Number of documents in train data: {}".format(len(text_train)))
print("Samples per class (train): {}".format(np.bincount(y_train)))

reviews_test = load_files("7374Assignment3/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target

print("Number of documents in test data: {}".format(len(text_test)))
print("Samples per class (test): {}".format(np.bincount(y_test)))

Number of documents in train data: 75000
Samples per class (train): [12500 12500 50000]
Number of documents in test data: 25000
Samples per class (test): [12500 12500]


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5, ngram_range=(2, 2))
x_train = vect.fit(text_train).transform(text_train)
x_test = vect.transform(text_test)

print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("X_train:\n{}".format(repr(x_train)))
print("X_test: \n{}".format(repr(x_test)))

feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Vocabulary size: 332885
X_train:
<75000x332885 sparse matrix of type '<class 'numpy.int64'>'
	with 12160550 stored elements in Compressed Sparse Row format>
X_test: 
<25000x332885 sparse matrix of type '<class 'numpy.int64'>'
	with 3852008 stored elements in Compressed Sparse Row format>
Number of features: 332885


In [0]:
batch_size = 32
epochs = 10

In [0]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [0]:
from keras import optimizers
# define network
model = Sequential()
model.add(Dense(50, input_shape=(x_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=['accuracy'])

In [43]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 75000 samples, validate on 25000 samples
Epoch 1/5
75000/75000 [==============================] - 306s 4ms/step - loss: 0.5189 - acc: 0.7730 - val_loss: 0.8323 - val_acc: 0.3350
Epoch 2/5
75000/75000 [==============================] - 305s 4ms/step - loss: 0.3975 - acc: 0.8057 - val_loss: 0.9210 - val_acc: 0.3579
Epoch 3/5
75000/75000 [==============================] - 303s 4ms/step - loss: 0.2962 - acc: 0.8665 - val_loss: 1.0559 - val_acc: 0.4012
Epoch 4/5
 8384/75000 [==>...........................] - ETA: 3:43 - loss: 0.2161 - acc: 0.9215

KeyboardInterrupt: ignored

In [54]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100) + '%')

Test Accuracy: 39.620001%


In [45]:
save_dir = os.path.join(os.getcwd(), '7374Assignment3')
model_name = 'keras_BOW_E2_model_v1.h5'
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/drive/My Drive/7374Assignment3/keras_BOW_E2_model_v1.h5 


In [47]:
# visualization
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epoch = range(len(acc))
 
plt.plot(epoch, acc, 'b', label='Training acc')
plt.plot(epoch, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epoch, loss, 'b', label='Training loss')
plt.plot(epoch, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

NameError: ignored

In [5]:
from keras.models import load_model

# Returns a compiled model identical to the previous one
model = load_model('7374Assignment3/keras_BOW_E2_model_v1.h5')

/content/drive/My Drive
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [0]:
with open('Final_test.json') as test_f:
   test_data = json.load(test_f)

In [0]:
test_data = pd.DataFrame(test_data)

In [8]:
test_data

,sentiment,text
0,Negative,"Moving on to taxes. As mentioned earlier, our ..."
1,Positive,Turning to Page 13 on Capital. Our Common Equi...
10,Neutral,[Operator Instructions]. Your first question i...
100,negative,"For my follow-up, Jen-Hsun you and Colette, yo..."
101,negative,"Well, in the short-term, in the near-term, we ..."
102,neutral,And your next question comes from the line of ...
103,negative,"Second, deteriorating macro economic condition..."
104,negative,I wanted to ask about again competition and da...
105,negative,Our data center business is really focused on ...
106,negative,If you think about competitively comparing our...


In [0]:
test_data['sentiment'] = test_data['sentiment'].str.lower()

In [0]:
test_data = test_data[test_data.sentiment != 'neutral']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5, ngram_range=(2, 2))
x_train = vect.fit(text_train).transform(text_train)
x_test = vect.transform(text_test)